In [1]:
import torch
from datasets import load_dataset
dataset = load_dataset('dair-ai/emotion')

from transformers import AutoTokenizer, AlbertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb",use_fast=False)



/home/ai/anaconda3/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
tokenizer

AlbertTokenizer(name_or_path='textattack/albert-base-v2-imdb', vocab_size=30000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [4]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [8]:
from transformers import  Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
model = AlbertForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb", num_labels=6,ignore_mismatched_sizes=True)
training_args = TrainingArguments(
output_dir="./results",
# Directory for saving outputs
learning_rate=5e-5,
# Learning rate for optimization
per_device_train_batch_size=16,
# Batch size for training
per_device_eval_batch_size=16,
# Batch size for evaluation
num_train_epochs=3,
# Number of training epochs
weight_decay=0.01,
# Weight decay for regularization
evaluation_strategy="epoch",
save_strategy = "no"
# Evaluation is done at the end of each epoch
)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at textattack/albert-base-v2-imdb and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [10]:
eval_results = trainer.evaluate(eval_dataset)
print(eval_results)

{'eval_loss': 1.802198052406311, 'eval_accuracy': 0.0975, 'eval_runtime': 34.9302, 'eval_samples_per_second': 57.257, 'eval_steps_per_second': 3.579}


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.239300,0.209393,0.921000
2,0.134600,0.160831,0.931000
3,0.092100,0.178811,0.929500


TrainOutput(global_step=3000, training_loss=0.2122759043375651, metrics={'train_runtime': 2277.1766, 'train_samples_per_second': 21.079, 'train_steps_per_second': 1.317, 'total_flos': 6358888710144000.0, 'train_loss': 0.2122759043375651, 'epoch': 3.0})

In [8]:
from ray import tune
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1,4 ,8 ,16 ]),
        "num_train_epochs": tune.choice([1,3,5]),

        "weight_decay": tune.loguniform(1e-4, 0.1)
    }

In [9]:
small_train_dataset = tokenized_datasets["train"].shard(index=1, num_shards=10) 
small_eval_dataset = tokenized_datasets["test"].shard(index=1, num_shards=10) 


In [10]:
num_labels = 6
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
def model_init():
    return  DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=6)

In [11]:




num_labels = 6
trainer = Trainer(

    model_init= model_init,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}

    

  
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
best_trial = trainer.hyperparameter_search(

    direction="maximize",

    backend="ray",

    hp_space=ray_hp_space,

    n_trials=10

)

/home/ai/anaconda3/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
2024-03-07 06:19:50,072	INFO worker.py:1724 -- Started a local Ray instance.
2024-03-07 06:19:51,726	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-07 06:19:51,729	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-03-07 06:19:51 (running for 00:00:00.18)
Using FIFO scheduling algorithm.
Logical resource usage: 0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_06-19-51
Number of trials: 10/10 (10 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |       |                 |                    |                ch_size |                |
|------------------------+----------+-------+-----------------+--------------------+------------------------+----------------|
| _objective_5ed05_00000 | PENDING  |       |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_5ed05_00001 | PENDING  |       |     1.57513e-05 |                  3

(_objective pid=77585) 2024-03-07 06:19:57.586483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_objective pid=77585) 2024-03-07 06:19:57.586532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_objective pid=77585) 2024-03-07 06:19:57.587850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_objective pid=77585) 2024-03-07 06:19:58.686151: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=77585) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased 

== Status ==
Current time: 2024-03-07 06:20:02 (running for 00:00:10.31)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_06-19-51
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                  |                 |                    |                ch_size |                |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------|
| _objective_5ed05_00000 | RUNNING  | 172.19.0.1:77585 |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_

  1%|          | 73/8000 [00:05<09:56, 13.29it/s]


...

  9%|▉         | 715/8000 [00:56<09:35, 12.65it/s]


== Status ==
Current time: 2024-03-07 06:20:57 (running for 00:01:05.70)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_06-19-51
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                  |                 |                    |                ch_size |                |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------|
| _objective_5ed05_00000 | RUNNING  | 172.19.0.1:77585 |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_

  9%|▉         | 733/8000 [00:57<09:43, 12.45it/s]
2024-03-07 06:20:59,083	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
  9%|▉         | 735/8000 [00:57<09:52, 12.25it/s]


== Status ==
Current time: 2024-03-07 06:20:59 (running for 00:01:07.32)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_06-19-51
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                  |                 |                    |                ch_size |                |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------------|
| _objective_5ed05_00000 | RUNNING  | 172.19.0.1:77585 |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_

 10%|█         | 803/8000 [01:03<09:48, 12.22it/s]

KeyboardInterrupt



In [14]:
best_trial

BestRun(run_id='0d889_00009', objective=0.89, hyperparameters={'learning_rate': 8.580760619921325e-06, 'per_device_train_batch_size': 1, 'num_train_epochs': 5, 'weight_decay': 0.010988100318524612}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x7fded96352d0>)

In [11]:

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

In [15]:
for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)



 19%|█▉        | 1559/8000 [02:06<08:10, 13.14it/s]


In [18]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.496800,0.423107,0.916000
2,0.258000,0.413957,0.922500
3,0.311600,0.391027,0.929000
4,0.229400,0.346086,0.927500
5,0.225400,0.472098,0.922000


TrainOutput(global_step=80000, training_loss=0.3704045206546783, metrics={'train_runtime': 9324.0715, 'train_samples_per_second': 8.58, 'train_steps_per_second': 8.58, 'total_flos': 1912604098560000.0, 'train_loss': 0.3704045206546783, 'epoch': 5.0})

In [14]:
trainer.save_model('/home/ai/Downloads/results/last-albert-emotion')

In [15]:
predictions = trainer.evaluate(train_dataset)
print(predictions)

{'eval_loss': 0.14407125115394592, 'eval_accuracy': 0.9651875, 'eval_runtime': 624.1207, 'eval_samples_per_second': 25.636, 'eval_steps_per_second': 1.602, 'epoch': 5.0}


In [16]:
predictions = trainer.evaluate(eval_dataset)
print(predictions)

{'eval_loss': 0.47209835052490234, 'eval_accuracy': 0.922, 'eval_runtime': 81.1871, 'eval_samples_per_second': 24.634, 'eval_steps_per_second': 1.54, 'epoch': 5.0}


In [17]:
predictions = trainer.evaluate(val_dataset)
print(predictions)

{'eval_loss': 0.36796051263809204, 'eval_accuracy': 0.939, 'eval_runtime': 85.1369, 'eval_samples_per_second': 23.492, 'eval_steps_per_second': 1.468, 'epoch': 5.0}


In [33]:
import random
import torch
num_examples = 10
picks = []
lable = []
input = []
prediction = [] 
for i in range(num_examples):
        pick = random.randint(0, len(dataset['validation'])-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
        input.append(dataset['validation'][picks[i]]['text'])
        #input_token = tokenizer(dataset['validation'][picks[i]]['text'])
        lable.append(dataset['validation'][picks[i]]['label'])
        encoded_input = tokenizer(dataset['validation'][pick]['text'],return_tensors='pt').to(model.device)
        prediction.append(torch.argmax(trainer.model.forward(**encoded_input).logits, dim=1)[0].cpu().item())
    
    
    


In [34]:
print(input)
print(lable,"lable")
print(prediction,"prediction")



['i eat or sleep i cant get myself to feel the life loving energy i felt so easily before', 'i love those kiddos and yet am left feeling so helpless', 'im just feeling personally devastated that this happened at my college in the school im studying under', 'im still feeling a little shocked over yesterdays news that pope benedict xvi has decided to resign', 'i want to feel pretty or handsome or something', 'i feel this way i do not just get to appreciate the amazing things i have right here and now i also get to dig up happy memories hidden back of my mind and i get to become inspired with hope for the future', 'i feel so terrified to tell her', 'i am feeling pretty relaxed though', 'i dont know whats wrong with me i try studying but i just feel like im fluffing around and getting distracted all the time', 'i am feeling much better and thought i should get going on the blogging front']
[2, 0, 0, 5, 1, 1, 4, 1, 3, 1] lable
[2, 4, 0, 5, 1, 1, 4, 1, 3, 1] prediction
